In [491]:
import os
import re
import pandas as pd

# Limpeza dos discursos

## 1 Criando uma lista com todos os discursos

In [492]:
discursos = []
for j in os.listdir('discursos'):
    file = open(f'discursos/{j}', 'r')
    text = file.read()
    discursos.append(text)
    file.close  


In [493]:
discursos = pd.Series(discursos)

In [494]:
#Criando lista com todos os conectivos da lingua portuguesa
conectivos = pd.read_html('https://duvidas.dicio.com.br/conectivos/')

a = conectivos[0]['Exemplos:']
conectivos_lista = []
for j in a:
    conectivos_lista.append(j.upper())
conectivos_lista = " ".join(conectivos_lista)

conectivos_lista = conectivos_lista.replace('; ', ' ').replace('… ', ' ').split(' ')
conectivos_lista = list(dict.fromkeys(conectivos_lista))
outros_conectivos = ['OS', 'E', 'UM', 'UMA']
for i in outros_conectivos:
    conectivos_lista.append(i)
conectivos_lista = sorted(conectivos_lista)

## 2 Verificando se houve revisão nos discursos

In [495]:
revisado = []
for j in range(len(discursos)):
    if(re.search(r'Sem revisão', discursos[j])):
        revisado.append(False)
    else:
        revisado.append(True)
revisao = pd.Series(revisado)

In [496]:
df_discursos = {"discursos" : discursos,
                "revisao" : revisao}
df_discursos = pd.concat(df_discursos, axis=1)

## 3 Limpando os discursos

In [500]:
from re import A
from urllib.request import AbstractBasicAuthHandler


df_discursos_limpos = []
for i in range(len(df_discursos['discursos'])):
    discurso = df_discursos['discursos'][i]

    # removendo a introdução
    discurso = discurso.replace((re.findall(r'(?<=^).*(?<=\))', discurso))[0], '')

    #Retirando os sons ambientes
    sons_ambiente = {'<tr><td align="center">(<i>Soa a campainha.</i>)</td></tr></table>', '<i>(Palmas.)</i>'}
    for j in sons_ambiente:
        discurso = discurso.replace(j, '') #Retirando os sons ambientes

    #Retirando as tags
    tags = {'<i>', '</i>', f'\n'}
    for j in tags:
        discurso = discurso.replace(j, '')

    #Retirando pontuacoes
    pontuacoes = {'.', ':', '...', '(', ')', '[', ']', '{', '}', '!', '?', ',', ';', '-', '_', "'", "\"", "nº", '<', '>', '\\', '|', '/'}
    for j in pontuacoes:
        discurso = discurso.replace(j, '')

    #retirando numeros
    for j in range(10):
        discurso = discurso.replace(str(j), '') 

    discurso =  discurso.split(' ')

    termos = []
    for j in range(len(discurso)):
        if discurso[j] != '':
            termos.append(discurso[j].upper())

    termos = {'termo' : pd.Series(sorted(termos))} 
    termos = pd.concat(termos, axis=1)
 

    for j in conectivos_lista:
        termos = termos[termos.termo != j]
    termos = termos.reset_index().drop(columns = ['index'])

    df_discursos_limpos.append(termos)

/tmp/ipykernel_27517/586308906.py:38: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  termos = {'termo' : pd.Series(sorted(termos))}
/tmp/ipykernel_27517/586308906.py:38: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  termos = {'termo' : pd.Series(sorted(termos))}
/tmp/ipykernel_27517/586308906.py:38: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  termos = {'termo' : pd.Series(sorted(termos))}
/tmp/ipykernel_27517/586308906.py:38: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  termos = {'termo' : pd.Series(sorted(termos))}


In [501]:
df_discursos_limpos[1]

,termo
0,ADMINISTRAÇÃO
1,BOLSONARO
2,BRASILEIRO
3,CIÊNCIA
4,DISSE
5,EDUCAÇÃO
6,EU
7,JAIR
8,NECESSÁRIOS
9,NOME


In [499]:
conectivos_lista

['A',
 'ACHO',
 'ACIMA',
 'ACORDO',
 'ADEMAIS',
 'ADMITO',
 'AGORA',
 'AINDA',
 'ALGUM',
 'ALGUMA',
 'ALIÁS',
 'ALTERNATIVA',
 'ALÉM',
 'ANALOGAMENTE',
 'ANTES',
 'AO',
 'APESAR',
 'APÓS',
 'AS',
 'ASSIM',
 'ATUALMENTE',
 'BEM',
 'CASO',
 'CAUSA',
 'CERTAMENTE',
 'CERTEZA',
 'CERTO',
 'COM',
 'COMEÇAR',
 'COMO',
 'CONCLUSÃO',
 'CONFORME',
 'CONFORMIDADE',
 'CONSEGUINTE',
 'CONSEQUÊNCIA',
 'CONSOANTE',
 'CONSTANTEMENTE',
 'CONSTATA-SE',
 'CONTANTO',
 'CONTRÁRIO',
 'CONTUDO',
 'CREIO',
 'CRER',
 'DA',
 'DE',
 'DECERTO',
 'DEPOIS',
 'DESDE',
 'DESSA',
 'DESSARTE',
 'DESSE',
 'DESTACA-SE',
 'DESTARTE',
 'DISSO',
 'DITO',
 'DIZER',
 'DO',
 'DÚVIDA',
 'E',
 'EFEITO',
 'EFETIVAMENTE',
 'EM',
 'EMBORA',
 'ENFIM',
 'ENQUANTO',
 'ENTANTO',
 'ENTENDER',
 'ENTRETANTO',
 'ENTÃO',
 'ESTOU',
 'EVENTUALMENTE',
 'EVIDENTEMENTE',
 'EXEMPLO',
 'FIM',
 'FINALIDADE',
 'FITO',
 'FORMA',
 'FREQUENTEMENTE',
 'HAJA',
 'HIPÓTESE',
 'HOJE',
 'IDÊNTICA',
 'IGUALMENTE',
 'IMEDIATAMENTE',
 'INESPERADAMENTE',
 'INOP